In [2]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder,OneHotEncoder,StandardScaler
import matplotlib.pyplot as plt
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.callbacks import ReduceLROnPlateau
from keras.layers import Flatten,Dropout
from keras.layers import BatchNormalization,Average
import collections

In [ ]:
#downloading dataset
!pip install requests
!pip install tqdm
import requests
!pip install wget
import wget
url = 'https://storage.googleapis.com/kaggle-competitions-data/kaggle/3004/train.csv?GoogleAccessId=web-data@kaggle-161607.iam.gserviceaccount.com&Expires=1541570583&Signature=BEHxUmIlbaz5dBiLA3wj8J%2FdxsL%2BrFmdhPcKMONq0ngFhQu1d1G8577KtGdzvo9dEPlqypIQp3x7w91216gdAYscj2x4MphYWv8vY6J0DzKVeegSIw6ahgM5nrivbC%2BDxV4H4w5eWEDitNR8mQZt8lQKCJF8FcciFkMFCHIIR1oLmI6YUjUwTZMR9ZzCm00Jd7O8wi1GXPP10k1pBY6BfD0sgFP5EKVIUPsn%2BiayA37v9xD7U3CmXaWIUb%2FcLB7281idcyUNQWMSKYQ9f5EtJ2D0P11w7s2nZVTBKhNGD3V9uZNF5LEPbuDtxxQenC3%2B7kxjwELIqEk8ddpc0HUU4Q%3D%3D'  
wget.download(url, './')
url = 'https://storage.googleapis.com/kaggle-competitions-data/kaggle/3004/test.csv?GoogleAccessId=web-data@kaggle-161607.iam.gserviceaccount.com&Expires=1541570649&Signature=DPqXGLD4zVsuTuh4xBHoAD3ZA0wuWCkmakSK2TUPpYAoLwibDIlmGSt3sH4fIfWjRmRG759ekxnjb%2BCUjJOnDhVfS5vi8iZKt3tStgovo4uYopb%2B2%2BCXsrp%2BK9fgYQ4lX1bLN6%2FUUSVsuv%2Fx7AW01OpqCMJ7wQZjyuWqjnGp18bwo0FtroV%2BX8iaa3Ea2vgddZDtYrN0DpZ5%2B6AGaKfxM1aw2ApHU2HC77HuYwyFwN4XuanzUpOc8PA2ri5wsTo5InPrz2DV6wWkwRl2FSR5KMBxKeLp9KfBnM3OamaOlg50DNVVL%2Fo2GYQsmUyFoifr1rD1T5qyNeTICP2R8G6g9g%3D%3D'  
wget.download(url, './')


In [3]:
train = pd.read_csv('./train.csv')
X = train.iloc[:,1:785].values
Y = train.iloc[:,0:1].values
X = X.reshape(42000,28,28,1)
onehotencoder = OneHotEncoder(categorical_features=[0])
Y = onehotencoder.fit_transform(Y).toarray()
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

In [ ]:
from keras.preprocessing.image import ImageDataGenerator
datagen = ImageDataGenerator(
    rotation_range=10)

In [ ]:
classifier = Sequential()
classifier.add(Conv2D(64,5,5,input_shape = (28,28,1),activation = 'relu'))
classifier.add(Conv2D(128,5,5,activation = 'relu'))
classifier.add(Dropout(0.2))
classifier.add(Conv2D(256,5,5,activation = 'relu'))
classifier.add(Conv2D(256,3,3,activation = 'relu'))
classifier.add(Dropout(0.25))
classifier.add(MaxPooling2D(pool_size=(2, 2)))
classifier.add(BatchNormalization())

classifier.add(Conv2D(512,3,3,activation = 'relu'))
classifier.add(Dropout(0.3))
classifier.add(BatchNormalization())
classifier.add(Conv2D(512,3,3,activation = 'relu'))
classifier.add(Dropout(0.4))
classifier.add(Flatten())
classifier.add(Dense(output_dim = 2000, init = 'uniform' , activation ='relu',input_dim = 784))
#classifier.add(Dense(output_dim = 1000, init = 'uniform' , activation ='relu'))
classifier.add(Dense(output_dim = 10, init = 'uniform' , activation ='softmax'))

#compiling model
classifier.compile(optimizer = 'adam',loss = 'categorical_crossentropy' , metrics = ['accuracy'])
reduce_lr = ReduceLROnPlateau(monitor='loss', factor=0.1,patience=1, min_lr=0.000000000001)
callbacks_list = [reduce_lr]
#fitting cnn to training set
classifier.fit_generator(datagen.flow(X, Y, batch_size=300),steps_per_epoch=len(X) / 300,nb_epoch = 40 ,callbacks = callbacks_list)

In [ ]:
classifier1 = Sequential()
classifier1.add(Conv2D(512,3,3,input_shape = (28,28,1),activation = 'relu'))
classifier1.add(Conv2D(256,5,5,activation = 'relu'))
classifier1.add(Conv2D(128,5,5,activation = 'relu'))
classifier1.add(Dropout(0.2))
classifier1.add(Conv2D(64,5,5,activation = 'relu'))
classifier1.add(Dropout(0.25))
classifier1.add(Conv2D(32,7,7,activation = 'relu'))
classifier1.add(MaxPooling2D(pool_size=(2, 2)))
classifier1.add(BatchNormalization())
classifier1.add(Dropout(0.3))
classifier1.add(Flatten())
classifier1.add(Dense(output_dim = 2000, init = 'uniform' , activation ='relu',input_dim = 784))
#classifier1.add(Dense(output_dim = 1000, init = 'uniform' , activation ='relu'))
classifier1.add(Dense(output_dim = 10, init = 'uniform' , activation ='softmax'))

#compiling model
classifier1.compile(optimizer = 'adam',loss = 'categorical_crossentropy' , metrics = ['accuracy'])
reduce_lr = ReduceLROnPlateau(monitor='loss', factor=0.1,patience=1, min_lr=0.0000000001)
callbacks_list = [reduce_lr]
#fitting cnn to training set
classifier1.fit_generator(datagen.flow(X,Y, batch_size=300),steps_per_epoch=len(X) / 300, nb_epoch = 40 ,callbacks = callbacks_list)

In [ ]:
from keras.preprocessing.image import ImageDataGenerator
datagen = ImageDataGenerator(
    rotation_range=10)
classifier2 = Sequential()
classifier2.add(Conv2D(8,3,3,input_shape = (28,28,1),activation = 'relu'))
classifier2.add(Conv2D(16,3,3,activation = 'relu'))
classifier2.add(Conv2D(32,3,3,activation = 'relu'))
classifier2.add(MaxPooling2D(pool_size=(2, 2)))
classifier2.add(BatchNormalization())
classifier2.add(Conv2D(64,3,3,activation = 'relu'))
classifier2.add(Conv2D(128,5,5,activation = 'relu'))
classifier2.add(Conv2D(256,3,3,activation = 'relu'))
classifier2.add(MaxPooling2D(pool_size=(2, 2)))
classifier2.add(BatchNormalization())

classifier2.add(Dropout(0.25))
classifier2.add(Conv2D(512,1,1,activation = 'relu'))
classifier2.add(Dropout(0.4))

classifier2.add(Flatten())

classifier2.add(Dense(output_dim = 2000, init = 'uniform' , activation ='relu',input_dim = 784))
classifier2.add(Dense(output_dim = 1000, init = 'uniform' , activation ='relu'))
classifier2.add(Dropout(0.50))
classifier2.add(BatchNormalization())
classifier2.add(Dense(output_dim = 10, init = 'uniform' , activation ='softmax'))
#compiling model
classifier2.compile(optimizer = 'adam',loss = 'categorical_crossentropy' , metrics = ['accuracy'])
reduce_lr = ReduceLROnPlateau(monitor='loss', factor=0.01,patience=1, min_lr=0.0000000000001)
callbacks_list = [reduce_lr]
#fitting cnn to training st
classifier2.fit_generator(datagen.flow(X,Y, batch_size=300),steps_per_epoch=len(X) / 300, nb_epoch = 40 ,callbacks = callbacks_list)

In [ ]:
from keras.preprocessing.image import ImageDataGenerator
datagen = ImageDataGenerator(
    rotation_range=10)
classifier3 = Sequential()
classifier3.add(Conv2D(8,3,3,input_shape = (28,28,1),activation = 'relu'))
classifier3.add(Conv2D(16,3,3,activation = 'relu'))
classifier3.add(Conv2D(32,3,3,activation = 'relu'))
classifier3.add(Conv2D(64,3,3,activation = 'relu'))
classifier3.add(Conv2D(128,5,5,activation = 'relu'))
classifier3.add(Conv2D(256,3,3,activation = 'relu'))
classifier3.add(BatchNormalization())

classifier3.add(Conv2D(512,3,3,activation = 'relu'))

classifier3.add(Flatten())

classifier3.add(Dense(output_dim = 2000, init = 'uniform' , activation ='relu',input_dim = 784))
classifier3.add(Dense(output_dim = 10, init = 'uniform' , activation ='softmax'))
#compiling model
classifier3.compile(optimizer = 'adam',loss = 'categorical_crossentropy' , metrics = ['accuracy'])
reduce_lr = ReduceLROnPlateau(monitor='loss', factor=0.01,patience=1, min_lr=0.0000000000001)
callbacks_list = [reduce_lr]
#fitting cnn to training st
classifier3.fit_generator(datagen.flow(X,Y, batch_size=200),steps_per_epoch=len(X) / 300, nb_epoch = 40 ,callbacks = callbacks_list)

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:5: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(8, (3, 3), input_shape=(28, 28, 1..., activation="relu")`
  """
/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:6: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(16, (3, 3), activation="relu")`
  
/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:7: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(32, (3, 3), activation="relu")`
  import sys
/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:8: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(64, (3, 3), activation="relu")`
  
/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:9: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(128, (5, 5), activation="relu")`
  if __name__ == '__main__':
/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:10: UserWarning: Update your `Conv2D` call 

Epoch 1/40
140/140 [==============================] - 23s 164ms/step - loss: 13.5714 - acc: 0.1534
Epoch 2/40
140/140 [==============================] - 16s 116ms/step - loss: 14.0836 - acc: 0.1262
Epoch 3/40
140/140 [==============================] - 16s 116ms/step - loss: 14.4412 - acc: 0.1040
Epoch 4/40
140/140 [==============================] - 16s 117ms/step - loss: 14.4332 - acc: 0.1045
Epoch 5/40
140/140 [==============================] - 16s 117ms/step - loss: 14.4706 - acc: 0.1022
Epoch 6/40
133/140 [===========================>..] - ETA: 0s - loss: 14.4596 - acc: 0.1029

In [ ]:
X_test = pd.read_csv('./test.csv')
X_test = X_test.iloc[:,:].values
X_test = X_test.reshape(28000,28,28,1)

In [ ]:
y_pred = classifier.predict(X_test)
for i in range(0,28000):
    for j in range(0,10):
        if(y_pred[i,j]<0.5):
            y_pred[i,j]=0
        if(y_pred[i,j]>0.5):
            y_pred[i,j]=1
for i in range(0,28000):
    for j in range(0,10):
        if(y_pred[i,j]==1):
            y_pred[i,0] = j
y_pred = np.delete(y_pred,np.s_[1:],axis = 1)
y_pred = y_pred.astype(int)

In [ ]:
y_pred1 = classifier1.predict(X_test)
for i in range(0,28000):
    for j in range(0,10):
        if(y_pred1[i,j]<0.5):
            y_pred1[i,j]=0
        if(y_pred1[i,j]>0.5):
            y_pred1[i,j]=1
for i in range(0,28000):
    for j in range(0,10):
        if(y_pred1[i,j]==1):
            y_pred1[i,0] = j
y_pred1 = np.delete(y_pred1,np.s_[1:],axis = 1)
y_pred1 = y_pred1.astype(int)

In [ ]:
y_pred2 = classifier2.predict(X_test)
for i in range(0,28000):
    for j in range(0,10):
        if(y_pred2[i,j]<0.5):
            y_pred2[i,j]=0
        if(y_pred2[i,j]>0.5):
            y_pred2[i,j]=1
for i in range(0,28000):
    for j in range(0,10):
        if(y_pred2[i,j]==1):
            y_pred2[i,0] = j
y_pred2 = np.delete(y_pred2,np.s_[1:],axis = 1)
y_pred2 = y_pred2.astype(int)

In [ ]:
for i in range(28000):
    outputs = [int(y_pred[i]),int(y_pred1[i]),int(y_pred2[i])]
    counter=collections.Counter(outputs)
    a = counter.most_common(1)
    if(a[0][1]>=2):
        y_pred[i] = a[0][0]
    else:
        y_pred[i] = y_pred1[i]

In [ ]:
data = pd.DataFrame(y_pred)
data.to_csv('output.csv')